In [12]:
try:
    spark.stop()
except:
    pass

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, round as spark_round
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType

from pyspark.ml.feature import StringIndexer, VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline
from pyspark.ml.functions import vector_to_array
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder \
    .appName("ALS") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.jars.packages", "org.apache.hudi:hudi-utilities-slim-bundle:0.13.0,org.apache.hudi:hudi-hive-sync:0.13.0") \
    .getOrCreate()


In [14]:
# Define schema
schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True)
])

# Load CSV
df = spark.read.csv("ratings.csv", header=True, schema=schema).cache()

# Optional: limit to first 1000 unique users for faster testing
unique_users = df.select("userId").distinct().limit(1000)
df = df.join(unique_users, on="userId", how="inner")

df.show(5)
print("Rows:", df.count())
print("Unique users:", df.select("userId").distinct().count())
print("Unique movies:", df.select("movieId").distinct().count())


+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|    31|     47|   5.0|
|    31|     50|   4.0|
|    31|    296|   4.5|
|    31|    318|   5.0|
|    31|    608|   4.5|
+------+-------+------+
only showing top 5 rows
Rows: 162988
Unique users: 1000
Unique movies: 12013


In [15]:
# Index users and movies
user_indexer = StringIndexer(inputCol="userId", outputCol="user")
movie_indexer = StringIndexer(inputCol="movieId", outputCol="movie")

# Assemble ratings into vector
assembler = VectorAssembler(inputCols=["rating"], outputCol="rating_vec")

# MinMax scale ratings to 0-1
scaler = MinMaxScaler(inputCol="rating_vec", outputCol="rating_scaled")

# Pipeline
pipeline = Pipeline(stages=[user_indexer, movie_indexer, assembler, scaler])
pipeline_model = pipeline.fit(df)
scaled_df = pipeline_model.transform(df)

# Flatten vector -> float, round to 1 decimal
scaled_df = scaled_df.withColumn(
    "rating_scaled",
    spark_round(vector_to_array(col("rating_scaled"))[0], 1)
)

# Keep only necessary columns
df_final = scaled_df.select("user", "movie", "rating_scaled")
df_final = df_final.withColumn("user", col("user").cast("int"))
df_final = df_final.withColumn("movie", col("movie").cast("int"))
df_final = df_final.withColumn("rating_scaled", col("rating_scaled").cast("float"))
df_final = df_final.dropna(subset=["user", "movie", "rating_scaled"])

df_final = df_final.cache()

df_final.show(5)


+----+-----+-------------+
|user|movie|rating_scaled|
+----+-----+-------------+
| 536|   24|          1.0|
| 536|   15|          0.8|
| 536|    4|          0.9|
| 536|    0|          1.0|
| 536|   22|          0.9|
+----+-----+-------------+
only showing top 5 rows


In [20]:
# Split df_final into 10 equal random splits
fractions = [0.5] * 2
splits = df_final.randomSplit(fractions, seed=42)

# Print split info
for i, part in enumerate(splits):
    n_users = part.select("user").distinct().count()
    n_movies = part.select("movie").distinct().count()
    print(f"Split {i+1}: {n_users} unique users, {n_movies} unique movies, {part.count()} rows")


Split 1: 1000 unique users, 9325 unique movies, 81755 rows
Split 2: 1000 unique users, 9294 unique movies, 81233 rows


In [6]:
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [21]:
all_models = []

for i, part in enumerate(splits, start=1):
    # Train/test split
    train_df, test_df = df_final.randomSplit([0.8, 0.2], seed=42)
    train_df.printSchema()
    train_df.show()

    
    # Initialize ALS
    als = ALS(
        userCol="user",
        itemCol="movie",
        ratingCol="rating_scaled",
        maxIter=20,
        regParam=0.1,
        rank=8,  # lower rank for small splits
        coldStartStrategy="drop",
        nonnegative=True
    )
    
    # Fit ALS model
    model = als.fit(train_df)
    
    # Predictions
    predictions = model.transform(test_df)
    
    # Evaluate RMSE
    evaluator = RegressionEvaluator(
        metricName="rmse",
        labelCol="rating_scaled",
        predictionCol="prediction"
    )
    
    rmse = evaluator.evaluate(predictions)
    print(f"Split {i}: Test RMSE = {rmse:.4f}")
    
    all_models.append(model)


root
 |-- user: integer (nullable = true)
 |-- movie: integer (nullable = true)
 |-- rating_scaled: float (nullable = true)

+----+-----+-------------+
|user|movie|rating_scaled|
+----+-----+-------------+
|   0|    0|          1.0|
|   0|    1|          0.8|
|   0|    3|          0.9|
|   0|    4|          1.0|
|   0|    5|          0.8|
|   0|    7|          1.0|
|   0|    9|          1.0|
|   0|   10|          1.0|
|   0|   11|          1.0|
|   0|   12|          0.6|
|   0|   14|          0.7|
|   0|   15|          1.0|
|   0|   16|          0.8|
|   0|   17|          0.8|
|   0|   18|          0.7|
|   0|   20|          0.9|
|   0|   21|          0.7|
|   0|   22|          1.0|
|   0|   24|          1.0|
|   0|   25|          0.7|
+----+-----+-------------+
only showing top 20 rows


Py4JJavaError: An error occurred while calling o1555.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task serialization failed: java.lang.StackOverflowError
java.lang.StackOverflowError
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1886)
	at java.base/java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1795)
	at java.base/java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1515)
	at java.base/java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1438)
	at java.base/java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1181)
	at java.base/java.io.ObjectOutputStream.defaultWriteFields(ObjectOutputStream.java:1572)
	at java.base/java.io.ObjectOutputStream.writeSerialData(ObjectOutputStream.java:1529)
	at java.base/java.io.ObjectOutputStream.writeOrdinaryObject(ObjectOutputStream.java:1438)
	at java.base/java.io.ObjectOutputStream